In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras import regularizers
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [42]:
data = pd.read_csv("datasets/movies.csv").drop(columns="judul_film")
data

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


In [45]:
# kategori = pd.get_dummies(data["genre"])
# data = pd.concat([data, kategori], axis=1)
# data.drop(columns="genre", inplace=True)
data

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


In [46]:
X = data['ringkasan_sinopsis'].values
y = data.iloc[:, 1:].values
X

array(['Raden Mas Said putra sulung Tumenggung Wilarikta di bawah Kerajaan Majapahit yang berkuasa di wilayah Tuban melihat sekeluarga miskin yang menderita busung lapar. Ia merasa sangat prihatin dan hati nuraninya tergugah untuk menolong. Kemudian ia mencoba secara diam-diam mengambil makanan dari lumbung orang tuanya. Perbuatan itu tidak disetujui orang tuanya bahkan ia dihukum sekap di gudang makanan itu. Sejak kejadian itu RM Said yang tumbuh dewasa tidak betah tinggal di rumah. Ia berkelana dari daerah satu ke daerah lainnya. Dari sanalah ia tahu betapa banyak penyelewengan dan kesewenang-wenangan para lurah yang munafik. Mereka selalu mengkambing-hitamkan Tumenggung untuk menutupi kejahatannya. Atas laporan RM Said ayahnya kemudian sadar. Tetapi kemudian ia dianggap sebagai sumber fitnah. Dalam kelananya kemudian ia bertemu dengan Sunan Bonang yang banyak mencurahkan ilmunya kepada RM Said. Ia pun kemudian melakukan tapa di pinggir kali. Berkat ketabahannya menghadapi berbagai c

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [63]:
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(X_train) 
tokenizer.fit_on_texts(X_test)
 
kalimat_train = tokenizer.texts_to_sequences(X_train)
kalimat_test = tokenizer.texts_to_sequences(X_test)
 
pading_train = pad_sequences(kalimat_train) 
pading_test = pad_sequences(kalimat_test)

pading_train

array([[   0,    0,    0, ..., 4275, 2203,  891],
       [   0,    0,    0, ...,    4,  125, 4643],
       [   0,    0,    0, ...,  501, 4643,  667],
       ...,
       [   0,    0,    0, ...,    4,  880,  563],
       [   0,    0,    0, ..., 1192,  782,  338],
       [   0,    0,    0, ..., 4643,  291, 3762]], dtype=int32)

# FUNGSI DROPOUT DAN REGULARIZERS  

In [66]:
# Fungsi Dropout dan regularizers mencegah overfitting model
model = Sequential([
    layers.Embedding(input_dim=5000, output_dim=32),
    layers.LSTM(64, kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.6),
    layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.6),
    layers.Dense(5, activation='softmax')
])


In [67]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
history = model.fit(pading_train, y_train, epochs=50, batch_size=128,
                    validation_data=(pading_test, y_test), verbose=2)

Epoch 1/50
7/7 - 4s - loss: 3.4951 - accuracy: 0.1965 - val_loss: 3.3754 - val_accuracy: 0.2090 - 4s/epoch - 609ms/step
Epoch 2/50
7/7 - 3s - loss: 3.3078 - accuracy: 0.1729 - val_loss: 3.1975 - val_accuracy: 0.2488 - 3s/epoch - 416ms/step
Epoch 3/50
7/7 - 3s - loss: 3.1352 - accuracy: 0.1903 - val_loss: 3.0357 - val_accuracy: 0.2139 - 3s/epoch - 409ms/step
Epoch 4/50
7/7 - 3s - loss: 2.9789 - accuracy: 0.1816 - val_loss: 2.8892 - val_accuracy: 0.1841 - 3s/epoch - 422ms/step
Epoch 5/50
7/7 - 3s - loss: 2.8371 - accuracy: 0.1978 - val_loss: 2.7570 - val_accuracy: 0.1692 - 3s/epoch - 416ms/step
Epoch 6/50
7/7 - 3s - loss: 2.7089 - accuracy: 0.2114 - val_loss: 2.6383 - val_accuracy: 0.1692 - 3s/epoch - 419ms/step
Epoch 7/50
7/7 - 3s - loss: 2.5946 - accuracy: 0.2090 - val_loss: 2.5319 - val_accuracy: 0.1692 - 3s/epoch - 424ms/step
Epoch 8/50
7/7 - 3s - loss: 2.4939 - accuracy: 0.1978 - val_loss: 2.4365 - val_accuracy: 0.1692 - 3s/epoch - 412ms/step
Epoch 9/50
7/7 - 3s - loss: 2.4010 - acc

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
plt.plot(history.history["loss"], label="train")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.plot(history.history["val_loss"], label="test");
plt.legend()

In [26]:
model.evaluate(pading_test,y_test)

7/7 [==============================] - 0s 38ms/step - loss: 1.6113 - accuracy: 0.1841


[1.6113343238830566, 0.1840796023607254]